In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score,RandomizedSearchCV,GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,precision_score,f1_score,recall_score,RocCurveDisplay

In [70]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [3]:
dataset = pd.read_csv("heart-disease.csv")
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [4]:
np.random.seed(40)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [37]:
sklearn_models = {
    "Logistic Regression":LogisticRegression(),
    "Random Forest":RandomForestClassifier(),
    "KNN":KNeighborsClassifier()
}

In [38]:
def fit_and_score(models,X_train,X_test,y_train,y_test):
    """
    Fits and evaluates given machine learning models.
    models: a dict of different scikit-learn machine learning models
    X_train : training data (no labels)
    X_test : testing data (no labels)
    y_train : training labels
    y_test : testing labels
    """
    np.random.seed(42)
    model_scores = {}
    for name,model in models.items():
        
        model.fit(X_train,y_train)
        model_scores[name] = model.score(X_test,y_test)
    return model_scores

In [39]:
model_scores = fit_and_score(models=sklearn_models,X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test)
model_scores

e:\Machine_Learning\end-to-end-heart_disease_classification\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Logistic Regression': 0.9016393442622951,
 'Random Forest': 0.819672131147541,
 'KNN': 0.7049180327868853}

# KNN

In [40]:
from sklearn.metrics import accuracy_score
train_scores=[]
test_scores=[]
acc_score=[]
neighbours = range(1,50)
for k in neighbours:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train,y_train)
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))
    acc_score.append(accuracy_score(y_test, knn.predict(X_test))*100)

In [41]:
print(f"Maximum score on test data is {max(test_scores)*100:.2f}")
print(f"Maximum accuracy_score is {max(acc_score):.2f}")

Maximum score on test data is 73.77
Maximum accuracy_score is 73.77


In [44]:
kf=KFold(n_splits=2,shuffle=True,random_state=42)
parameter={'n_neighbors': np.arange(1, 50)}
knn=KNeighborsClassifier()
knn_cv=GridSearchCV(knn, param_grid=parameter, cv=kf, verbose=1)
knn_cv.fit(X_train, y_train)
print(knn_cv.best_params_)

Fitting 2 folds for each of 49 candidates, totalling 98 fits
{'n_neighbors': np.int64(5)}


In [87]:
k = knn_cv.best_params_
k

{'n_neighbors': np.int64(5)}

In [45]:
best_k = int(knn_cv.best_params_['n_neighbors'])  # convert np.int64 to int
final_knn = KNeighborsClassifier(n_neighbors=best_k)
final_knn.fit(X_train, y_train)

train_acc = final_knn.score(X_train, y_train)
test_acc = final_knn.score(X_test, y_test)

print(f"Train Accuracy: {train_acc:.2%}")
print(f"Test Accuracy: {test_acc:.2%}")


Train Accuracy: 76.86%
Test Accuracy: 70.49%


In [83]:
k_acc = knn_cv.best_score_

In [84]:
k_test = knn_cv.score(X_test, y_test)

In [82]:
best_k

5

# RANDOM FOREST

In [51]:
from sklearn.metrics import accuracy_score
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, None],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt']
}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.score(X_test, y_test)
accuracy_score(y_test, grid_search.predict(X_test))*100

83.60655737704919

In [48]:
grid_search.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_split': 5,
 'n_estimators': 200}

# LOGISITC REGRESSSION

In [52]:
parameters = {
    'penalty' : ['l1','l2'], 
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}

In [64]:
logreg = LogisticRegression()
clf = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='accuracy',        # metric for scoring
                   cv=2,verbose=1)          

In [65]:
clf.fit(X_train,y_train)

Fitting 2 folds for each of 42 candidates, totalling 84 fits


e:\Machine_Learning\end-to-end-heart_disease_classification\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\Machine_Learning\end-to-end-heart_disease_classification\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown

,estimator,LogisticRegression()
,param_grid,"{'C': array([1.e-03...e+02, 1.e+03]), 'penalty': ['l1', 'l2'], 'solver': ['newton-cg', 'lbfgs', ...]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,2
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [66]:
clf.score(X_test, y_test)

0.9016393442622951

In [67]:
print("Tuned Hyperparameters :", clf.best_params_)
print("Accuracy :",clf.best_score_)

Tuned Hyperparameters : {'C': np.float64(1.0), 'penalty': 'l2', 'solver': 'newton-cg'}
Accuracy : 0.8305785123966942


In [69]:
logreg = LogisticRegression(C = 1.0, 
                            penalty = 'l2', 
                            solver = 'newton-cg')
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print("Accuracy:",logreg.score(X_test, y_test))

Accuracy: 0.9016393442622951


In [79]:
bscore = clf.best_score_

In [80]:
bscore

np.float64(0.8305785123966942)

In [77]:
test_accuracy = clf.score(X_test, y_test)

In [78]:
test_accuracy

0.9016393442622951

In [74]:
log_params = clf.best_params_
log_params

{'C': np.float64(1.0), 'penalty': 'l2', 'solver': 'newton-cg'}

In [71]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("classifier", LogisticRegression())])
param_grid = [
    # Logistic Regression
    {
        "classifier": [LogisticRegression(max_iter=1000)],
        "classifier__penalty": ["l1", "l2"],
        "classifier__C": np.logspace(-3, 3, 7),
        "classifier__solver": ["liblinear", "lbfgs", "newton-cg"]
    },
    # Random Forest
    {
        "classifier": [RandomForestClassifier()],
        "classifier__n_estimators": [100, 200],
        "classifier__max_depth": [10, None],
        "classifier__min_samples_split": [2, 5],
        "classifier__max_features": ["sqrt"]
    },
    # KNN
    {
        "classifier": [KNeighborsClassifier()],
        "classifier__n_neighbors": np.arange(1, 50)
    }
]


grid = GridSearchCV(pipe, param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

# Get best model details
best_model = grid.best_estimator_
best_params = grid.best_params_
best_cv_score = grid.best_score_

# Evaluate on test set
y_pred = best_model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)

Fitting 5 folds for each of 99 candidates, totalling 495 fits


e:\Machine_Learning\end-to-end-heart_disease_classification\.venv\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
70 fits failed out of a total of 495.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Machine_Learning\end-to-end-heart_disease_classification\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Machine_Learning\end-to-end-heart_disease_classification\.venv\lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "e:\Machine_Learning\end-to-end-heart_

In [88]:
mlflow.set_experiment("Heart Disease Classification")
with mlflow.start_run(run_name="K-Neighbors Classifier"):
    mlflow.log_params(k)  # all best hyperparameters
    mlflow.log_metric("cv_accuracy", k_acc)
    mlflow.log_metric("test_accuracy",k_test )
    mlflow.sklearn.log_model(knn_cv, "best_model")
print("Best model logged to MLflow:")
print(knn_cv)

2025/08/14 14:50:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/14 14:50:45 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/08/14 14:50:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best model logged to MLflow:
GridSearchCV(cv=KFold(n_splits=2, random_state=42, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])},
             verbose=1)
